In [ ]:
from networks.unet_leiterrl import weights_init, TurbNetG, UNet
from networks.dummy_network import DummyNet
from torch.utils.tensorboard import SummaryWriter
from torch.nn import MSELoss
import main

# autoreload
%load_ext autoreload
%autoreload 2

# Training

In [ ]:
inputs="pk"
# parameters of model and training
loss_fn = MSELoss()
n_epochs = 1 #00 #60000

unet_model = UNet(in_channels=len(inputs)+1, out_channels=1).float()
# fc_model = DummyNet().float()
# model.to(device)

name_folder="try_unstructured_grid" 

learning_rate=5e-4 #0.0005 #0.0004

In [ ]:
main.run_experiment(n_epochs=1, dataset_name="perm_pressure1D_100dp_domainlarge/perm_pressure_100dp");